In [ ]:
import copy
import seaborn as sns
from seaborn.utils import np, pd, plt, os

In [ ]:
my_pkg = "..//Presentation"
imp_mods = os.sys.path 
if my_pkg not in imp_mods:
    imp_mods.append(my_pkg)
    print("Imported successfully")

In [ ]:
from census_methods import CensusDataset as cd 

In [ ]:
census_df = pd.read_csv('final_Marital_Status_and_Religion_cleaned.csv')

In [ ]:
census_df.info()

#### BIRTH RATE

In [ ]:
# Crude Birth Rate = (birth_thisyr / total_population) * 1000
birth_thisyr = census_df.loc[census_df['Age'] == 0].shape[0]
population = census_df.shape[0]
cbr = 1000 * birth_thisyr/population
print(f"There are {cbr:.2f} births per thousand residents")

#### CHANGES IN BIRTH RATE

In [ ]:
# 4yr_br - cbr
babies_4yr = census_df.loc[census_df['Age'] == 4].shape[0]
br_4yr = 1000 * babies_4yr/population
br_diff = cbr - br_4yr
print(f"Birth rate 4 years ago was {br_4yr:.2f} births per thousand residents")

In [ ]:
print(f"Current birth rate change from previous per 1000: {br_diff:.2f}")

#### AGE SPECIFIC FERTILITY RATE

In [ ]:
# 25 to 29 yo
mothers2529 = census_df.loc[(census_df['Age'].between(25, 29)) &
                           (census_df['Gender'] == 'F')].shape[0]

asfr25 = 1000 * (birth_thisyr/mothers2529)
asfr25

In [ ]:
# 30 to 34 yo
mothers3034 = census_df.loc[(census_df['Age'].between(30, 34)) &
                           (census_df['Gender'] == 'F')].shape[0]
asfr30 = 1000 * (babies_4yr/mothers3034)
asfr30

In [ ]:
# 35 to 39 yo
mothers3539 = census_df.loc[(census_df['Age'].between(35, 39)) &
                           (census_df['Gender'] == 'F')].shape[0]

babies_8yr = census_df.loc[census_df['Age'] == 8].shape[0]

asfr35 = 1000 * (babies_8yr/mothers3539)
asfr35

In [ ]:
print(f"Age Specific Fertility Rate 25-29: {asfr25}")
print(f"Age Specific Fertility Rate 30-34: {asfr30}")
print(f"Age Specific Fertility Rate 35-39: {asfr35}")

#### GENERAL FERTILITY RATE

In [ ]:
sum_babies = birth_thisyr + babies_4yr + babies_8yr
sum_mothers = mothers2529 + mothers3034 + mothers3539
gen_fert_rate = 1000 * (sum_babies/sum_mothers)
print(f"General fertility rate: {gen_fert_rate}")

#### CRUDE DEATH RATE

In [ ]:
senior_citizens = census_df.loc[census_df['Age'] >= 65]
senior_age_freq = senior_citizens['Age'].value_counts().sort_index(ascending=False)
senior_age_freq.name = 'Age Count'
senior_age_freq

In [ ]:
# estimate last year's ages
lastyr_senior_freq = senior_age_freq.shift(-1)
lastyr_senior_freq

In [ ]:
diff_btw_seniors = senior_age_freq - lastyr_senior_freq
diff_btw_seniors

In [ ]:
deaths = diff_btw_seniors.loc[diff_btw_seniors < 0]
deaths

In [ ]:
death_rate = 1000 * np.abs(deaths).sum()/population
print(f"Town's death rate: {death_rate} deaths per 1000 resident")

#### CRUDE GROWTH RATE

In [ ]:
cgr = cbr - death_rate
print(f"Crude Growth Rate: {cgr} per 1000 resident")
print(f"Percentage Crude Growth Rate: {cgr/10}")

In [ ]:
sns.set_style('darkgrid')
fig = plt.figure(figsize=(8, 6), dpi=200)

sns.barplot(x=deaths, y=deaths.index, color='red', ci=None)
sns.barplot(x=diff_btw_seniors.loc[~diff_btw_seniors.index.isin(deaths.index)],
           y=diff_btw_seniors.loc[~diff_btw_seniors.index.isin(deaths.index)].index,
           color='green', ci=None)
plt.title("Changes in Population Among 65+ Year-Olds")